# LangGraph 2.1: Single Agent Development

## Complete Guide with Executable Examples

### Topics Covered:
1. **ReAct Pattern** (Reasoning and Acting)
2. **Agent Nodes and Tool Integration**
3. **Function Calling and Tool Execution**
4. **Error Handling and Retries**
5. **Basic Debugging Techniques**

## Setup and Installation

First, install the required packages:

In [3]:
# Install required packages
!pip install langgraph langchain-anthropic langchain-core -q

In [4]:
# Import required libraries
import os
from typing import Annotated, Literal, TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage, ToolMessage
from langchain_core.tools import tool
from langchain_anthropic import ChatAnthropic

# Set your API key
from dotenv import load_dotenv
load_dotenv()

print("✅ Imports completed successfully!")

✅ Imports completed successfully!


---

# 1. ReAct Pattern - Reasoning and Acting

## What is ReAct?

ReAct is a paradigm where an agent alternates between:
- **Reasoning**: Thinking about what to do next
- **Acting**: Executing tools/actions based on reasoning
- **Observing**: Processing the results of actions

```
User Query → Reasoning → Action → Observation → Reasoning → Action → ... → Answer
```

### Benefits:
- More interpretable agent behavior
- Better error recovery
- Reduced hallucination through grounding in tool results
- Natural chain-of-thought reasoning

## 1.1 Define Simple Tools

In [7]:
@tool
def calculate(operation: str, a: float, b: float) -> float:
    """
    Perform basic mathematical operations.
    
    Args:
        operation: The operation to perform (add, subtract, multiply, divide)
        a: First number
        b: Second number
    
    Returns:
        Result of the operation
    """
    operations = {
        "add": lambda x, y: x + y,
        "subtract": lambda x, y: x - y,
        "multiply": lambda x, y: x * y,
        "divide": lambda x, y: x / y if y != 0 else "Error: Division by zero"
    }
    
    if operation not in operations:
        return f"Error: Unknown operation {operation}"
    
    result = operations[operation](a, b)
    print(f"🔧 Tool executed: {operation}({a}, {b}) = {result}")
    return result


@tool
def get_weather(location: str) -> str:
    """
    Get weather information for a location.
    
    Args:
        location: City name or location
    
    Returns:
        Weather information
    """
    # Simulated weather data
    weather_data = {
        "san francisco": "Sunny, 68°F",
        "new york": "Cloudy, 55°F",
        "london": "Rainy, 52°F",
        "tokyo": "Clear, 72°F"
    }
    
    location_lower = location.lower()
    weather = weather_data.get(location_lower, f"Weather data not available for {location}")
    print(f"🔧 Tool executed: get_weather('{location}') = {weather}")
    return weather

print("✅ Tools defined: calculate, get_weather")

✅ Tools defined: calculate, get_weather


## 1.2 Define Agent State

In [9]:
class AgentState(TypedDict):
    """State of the agent including conversation history"""
    messages: Annotated[list, add_messages]

print("✅ AgentState defined")

✅ AgentState defined


## 1.3 Create ReAct Agent

In [11]:
def create_react_agent():
    """Create a ReAct agent with reasoning and acting capabilities"""
    
    # Initialize the LLM with tools
    tools = [calculate, get_weather]
    llm = ChatAnthropic(model="claude-sonnet-4-20250514", temperature=0)
    llm_with_tools = llm.bind_tools(tools)
    
    # Define the agent node (reasoning)
    def agent_node(state: AgentState):
        """Agent decides what to do next based on the current state"""
        print("\n🧠 REASONING: Agent is thinking...")
        response = llm_with_tools.invoke(state["messages"])
        print(f"   Agent decision: {'Use tool' if response.tool_calls else 'Respond directly'}")
        return {"messages": [response]}
    
    # Build the graph
    workflow = StateGraph(AgentState)
    
    # Add nodes
    workflow.add_node("agent", agent_node)
    workflow.add_node("tools", ToolNode(tools))
    
    # Add edges
    workflow.add_edge(START, "agent")
    workflow.add_conditional_edges(
        "agent",
        tools_condition,  # Automatically routes based on tool calls
    )
    workflow.add_edge("tools", "agent")
    
    # Compile the graph
    memory = MemorySaver()
    app = workflow.compile(checkpointer=memory)
    
    return app

print("✅ create_react_agent function defined")

✅ create_react_agent function defined


## 1.4 Test ReAct Agent

In [13]:
# Create the agent
react_agent = create_react_agent()

# Test query requiring reasoning and acting
query = "What is 15 multiplied by 8, and what's the weather in Tokyo?"

print(f"📝 Query: {query}")
print("=" * 80)

result = react_agent.invoke(
    {"messages": [HumanMessage(content=query)]},
    config={"configurable": {"thread_id": "react_example_1"}}
)

print("\n" + "=" * 80)
print(f"✅ Final Answer: {result['messages'][-1].content}")

📝 Query: What is 15 multiplied by 8, and what's the weather in Tokyo?

🧠 REASONING: Agent is thinking...
   Agent decision: Use tool
🔧 Tool executed: multiply(15.0, 8.0) = 120.0
🔧 Tool executed: get_weather('Tokyo') = Clear, 72°F

🧠 REASONING: Agent is thinking...
   Agent decision: Respond directly

✅ Final Answer: Here are your answers:

1. **15 multiplied by 8 = 120**

2. **Weather in Tokyo: Clear, 72°F**

The weather in Tokyo is currently clear with a temperature of 72°F (about 22°C).


---

# 2. Agent Nodes and Tool Integration

## Advanced tool integration with custom node logic

Let's create more sophisticated tools and track agent behavior.

## 2.1 Define Advanced Tools

In [16]:
@tool
def search_database(query: str) -> str:
    """
    Search a database for information.
    
    Args:
        query: Search query
    
    Returns:
        Search results
    """
    # Simulated database
    database = {
        "python": "Python is a high-level programming language.",
        "langgraph": "LangGraph is a library for building stateful agents.",
        "react": "ReAct is a pattern for reasoning and acting."
    }
    
    result = database.get(query.lower(), f"No results found for '{query}'")
    print(f"🔧 Tool: search_database('{query}') → {result}")
    return result


@tool
def save_to_file(filename: str, content: str) -> str:
    """
    Save content to a file.
    
    Args:
        filename: Name of the file
        content: Content to save
    
    Returns:
        Success message
    """
    # Simulated file save
    print(f"🔧 Tool: save_to_file('{filename}', content_length={len(content)})")
    return f"Successfully saved {len(content)} characters to {filename}"

print("✅ Advanced tools defined")

✅ Advanced tools defined


## 2.2 Enhanced Agent State with Metadata

In [18]:
class EnhancedAgentState(TypedDict):
    """Enhanced state with metadata"""
    messages: Annotated[list, add_messages]
    iteration_count: int
    tool_calls_made: list

print("✅ EnhancedAgentState defined")

✅ EnhancedAgentState defined


## 2.3 Create Enhanced Agent with Tracking

In [20]:
def create_enhanced_agent():
    """Create an agent with custom node logic and multiple tools"""
    
    tools = [calculate, get_weather, search_database, save_to_file]
    llm = ChatAnthropic(model="claude-sonnet-4-20250514", temperature=0)
    llm_with_tools = llm.bind_tools(tools)
    
    # Custom agent node with metadata tracking
    def custom_agent_node(state: EnhancedAgentState):
        """Agent with enhanced tracking"""
        print(f"\n🧠 Agent Iteration #{state.get('iteration_count', 0) + 1}")
        
        # Add system message for better reasoning
        messages = state["messages"]
        if not any(isinstance(m, SystemMessage) for m in messages):
            messages = [
                SystemMessage(content="You are a helpful assistant. Think step-by-step.")
            ] + messages
        
        response = llm_with_tools.invoke(messages)
        
        # Track tool calls
        tool_calls_made = state.get("tool_calls_made", [])
        if response.tool_calls:
            for tc in response.tool_calls:
                tool_calls_made.append(tc["name"])
        
        return {
            "messages": [response],
            "iteration_count": state.get("iteration_count", 0) + 1,
            "tool_calls_made": tool_calls_made
        }
    
    # Build graph
    workflow = StateGraph(EnhancedAgentState)
    workflow.add_node("agent", custom_agent_node)
    workflow.add_node("tools", ToolNode(tools))
    
    workflow.add_edge(START, "agent")
    workflow.add_conditional_edges("agent", tools_condition)
    workflow.add_edge("tools", "agent")
    
    return workflow.compile(checkpointer=MemorySaver())

print("✅ create_enhanced_agent function defined")

✅ create_enhanced_agent function defined


## 2.4 Test Enhanced Agent

In [22]:
enhanced_agent = create_enhanced_agent()

query = "Search for 'langgraph', then calculate 25 plus 17"

print(f"📝 Query: {query}")
print("=" * 80)

result = enhanced_agent.invoke(
    {"messages": [HumanMessage(content=query)], "iteration_count": 0, "tool_calls_made": []},
    config={"configurable": {"thread_id": "enhanced_1"}}
)

print("\n" + "=" * 80)
print(f"📊 Metadata:")
print(f"   Iterations: {result['iteration_count']}")
print(f"   Tools used: {result['tool_calls_made']}")
print(f"\n✅ Final Answer: {result['messages'][-1].content}")

📝 Query: Search for 'langgraph', then calculate 25 plus 17

🧠 Agent Iteration #1
🔧 Tool: search_database('langgraph') → LangGraph is a library for building stateful agents.
🔧 Tool executed: add(25.0, 17.0) = 42.0

🧠 Agent Iteration #2

📊 Metadata:
   Iterations: 2
   Tools used: ['search_database', 'calculate']

✅ Final Answer: Here are the results:

**Search results for 'langgraph':**
LangGraph is a library for building stateful agents.

**Calculation:**
25 + 17 = 42


---

# 3. Function Calling and Tool Execution

## Understanding how tools are called and executed

Let's explore complex tool parameters and custom execution logic.

## 3.1 Tool with Complex Parameters

In [25]:
@tool
def create_report(
    title: str,
    sections: list[str],
    include_summary: bool = True
) -> str:
    """
    Create a structured report.
    
    Args:
        title: Report title
        sections: List of section names
        include_summary: Whether to include a summary
    
    Returns:
        Formatted report
    """
    report = f"# {title}\n\n"
    
    for i, section in enumerate(sections, 1):
        report += f"## {i}. {section}\n[Content for {section}]\n\n"
    
    if include_summary:
        report += "## Summary\n[Report summary]\n"
    
    print(f"🔧 Tool: create_report(title='{title}', sections={len(sections)})")
    return report

print("✅ create_report tool defined")

✅ create_report tool defined


## 3.2 Custom Tool Executor with Logging

In [27]:
class ToolExecutor:
    """Custom tool executor with detailed logging"""
    
    def __init__(self, tools):
        self.tools = {tool.name: tool for tool in tools}
        self.execution_log = []
    
    def execute(self, tool_name: str, tool_input: dict):
        """Execute a tool and log the execution"""
        print(f"\n⚙️  Executing: {tool_name}")
        print(f"   Input: {tool_input}")
        
        if tool_name not in self.tools:
            error = f"Tool '{tool_name}' not found"
            print(f"   ❌ Error: {error}")
            return {"error": error}
        
        try:
            tool = self.tools[tool_name]
            result = tool.invoke(tool_input)
            
            self.execution_log.append({
                "tool": tool_name,
                "input": tool_input,
                "output": result,
                "status": "success"
            })
            
            print(f"   ✅ Success")
            return result
            
        except Exception as e:
            error_msg = str(e)
            print(f"   ❌ Error: {error_msg}")
            
            self.execution_log.append({
                "tool": tool_name,
                "input": tool_input,
                "error": error_msg,
                "status": "failed"
            })
            
            return {"error": error_msg}
    
    def get_log(self):
        """Get execution log"""
        return self.execution_log

print("✅ ToolExecutor class defined")

✅ ToolExecutor class defined


## 3.3 Test Tool Executor

In [29]:
executor = ToolExecutor([calculate, create_report])

print("📝 Testing Tool Executor")
print("=" * 80)

# Execute tools
executor.execute("calculate", {"operation": "multiply", "a": 12, "b": 5})
executor.execute("create_report", {
    "title": "Q4 Report",
    "sections": ["Introduction", "Results", "Conclusion"],
    "include_summary": True
})
executor.execute("nonexistent_tool", {})  # Error case

print("\n" + "=" * 80)
print("📋 Execution Log:")
for i, log_entry in enumerate(executor.get_log(), 1):
    print(f"\n   {i}. {log_entry['tool']}: {log_entry['status']}")

📝 Testing Tool Executor

⚙️  Executing: calculate
   Input: {'operation': 'multiply', 'a': 12, 'b': 5}
🔧 Tool executed: multiply(12.0, 5.0) = 60.0
   ✅ Success

⚙️  Executing: create_report
   Input: {'title': 'Q4 Report', 'sections': ['Introduction', 'Results', 'Conclusion'], 'include_summary': True}
🔧 Tool: create_report(title='Q4 Report', sections=3)
   ✅ Success

⚙️  Executing: nonexistent_tool
   Input: {}
   ❌ Error: Tool 'nonexistent_tool' not found

📋 Execution Log:

   1. calculate: success

   2. create_report: success


---

# 4. Error Handling and Retries

## Building robust agents that can handle failures

Real-world applications need to handle:
- Network failures
- API timeouts
- Invalid inputs
- Tool errors

## 4.1 Simulated Unreliable Tool

In [32]:
from time import sleep

@tool
def unreliable_api(data: str, fail_probability: float = 0.5) -> str:
    """
    Simulates an unreliable API that might fail.
    
    Args:
        data: Data to process
        fail_probability: Probability of failure (0-1)
    
    Returns:
        Processed data or error
    """
    import random
    
    if random.random() < fail_probability:
        raise Exception("API temporarily unavailable")
    
    return f"Processed: {data}"

print("✅ unreliable_api tool defined")

✅ unreliable_api tool defined


## 4.2 Agent State with Retry Tracking

In [34]:
class RetryableAgentState(TypedDict):
    """State with retry tracking"""
    messages: Annotated[list, add_messages]
    retry_count: int
    max_retries: int
    last_error: str | None

print("✅ RetryableAgentState defined")

✅ RetryableAgentState defined


## 4.3 Create Agent with Retry Logic

In [36]:
def create_agent_with_retry():
    """Create an agent with built-in retry logic"""
    
    tools = [calculate, unreliable_api]
    llm = ChatAnthropic(model="claude-sonnet-4-20250514", temperature=0)
    llm_with_tools = llm.bind_tools(tools)
    
    def agent_with_error_handling(state: RetryableAgentState):
        """Agent node with error awareness"""
        print(f"\n🧠 Agent (Retry count: {state.get('retry_count', 0)})")
        
        messages = state["messages"]
        
        # If there was an error, inform the agent
        if state.get("last_error"):
            error_msg = f"Previous action failed with error: {state['last_error']}. Try a different approach."
            messages = messages + [SystemMessage(content=error_msg)]
            print(f"   ⚠️  Informing agent of error: {state['last_error']}")
        
        response = llm_with_tools.invoke(messages)
        
        return {
            "messages": [response],
            "last_error": None  # Clear error after processing
        }
    
    def tools_with_retry(state: RetryableAgentState):
        """Tool execution with retry logic"""
        max_retries = state.get("max_retries", 3)
        retry_count = state.get("retry_count", 0)
        
        last_message = state["messages"][-1]
        
        for tool_call in last_message.tool_calls:
            attempt = 0
            while attempt < max_retries:
                try:
                    print(f"\n⚙️  Executing {tool_call['name']} (attempt {attempt + 1}/{max_retries})")
                    
                    # Execute tool
                    tool = next(t for t in tools if t.name == tool_call["name"])
                    result = tool.invoke(tool_call["args"])
                    
                    print(f"   ✅ Success!")
                    
                    # Create tool message
                    return {
                        "messages": [ToolMessage(
                            content=str(result),
                            tool_call_id=tool_call["id"]
                        )],
                        "retry_count": 0,  # Reset on success
                        "last_error": None
                    }
                    
                except Exception as e:
                    attempt += 1
                    error_msg = str(e)
                    print(f"   ❌ Failed: {error_msg}")
                    
                    if attempt < max_retries:
                        print(f"   🔄 Retrying in 1 second...")
                        sleep(1)
                    else:
                        print(f"   ❌ Max retries reached")
                        return {
                            "messages": [ToolMessage(
                                content=f"Error after {max_retries} attempts: {error_msg}",
                                tool_call_id=tool_call["id"]
                            )],
                            "retry_count": retry_count + 1,
                            "last_error": error_msg
                        }
        
        return {"retry_count": 0, "last_error": None}
    
    # Build graph
    workflow = StateGraph(RetryableAgentState)
    workflow.add_node("agent", agent_with_error_handling)
    workflow.add_node("tools", tools_with_retry)
    
    workflow.add_edge(START, "agent")
    workflow.add_conditional_edges("agent", tools_condition)
    workflow.add_edge("tools", "agent")
    
    return workflow.compile(checkpointer=MemorySaver())

print("✅ create_agent_with_retry function defined")

✅ create_agent_with_retry function defined


## 4.4 Test Retry Logic

In [38]:
retry_agent = create_agent_with_retry()

print("📝 Testing Error Handling with Retries")
print("=" * 80)

# This might fail and retry
result = retry_agent.invoke(
    {
        "messages": [HumanMessage(content="Calculate 10 + 5")],
        "retry_count": 0,
        "max_retries": 3,
        "last_error": None
    },
    config={"configurable": {"thread_id": "retry_1"}}
)

print("\n" + "=" * 80)
print(f"✅ Final result received")

📝 Testing Error Handling with Retries

🧠 Agent (Retry count: 0)

⚙️  Executing calculate (attempt 1/3)
🔧 Tool executed: add(10.0, 5.0) = 15.0
   ✅ Success!

🧠 Agent (Retry count: 0)

✅ Final result received


---

# 5. Basic Debugging Techniques

## Monitoring and debugging agent behavior

Effective debugging is crucial for:
- Understanding agent decisions
- Identifying bottlenecks
- Catching errors early
- Optimizing performance

## 5.1 Debug Agent State

In [41]:
class DebugAgentState(TypedDict):
    """State with debug information"""
    messages: Annotated[list, add_messages]
    debug_log: list

print("✅ DebugAgentState defined")

✅ DebugAgentState defined


## 5.2 Create Debuggable Agent

In [43]:
def create_debuggable_agent():
    """Create an agent with comprehensive debugging"""
    
    tools = [calculate, get_weather]
    llm = ChatAnthropic(model="claude-sonnet-4-20250514", temperature=0)
    llm_with_tools = llm.bind_tools(tools)
    
    def debug_agent_node(state: DebugAgentState):
        """Agent node with debug logging"""
        debug_log = state.get("debug_log", [])
        
        print("\n" + "=" * 60)
        print("🐛 DEBUG: Agent Node Execution")
        print("=" * 60)
        
        # Log current state
        print(f"Messages in state: {len(state['messages'])}")
        print(f"Last message: {state['messages'][-1].content[:100]}...")
        
        debug_log.append({
            "node": "agent",
            "message_count": len(state['messages']),
            "last_message_type": type(state['messages'][-1]).__name__
        })
        
        # Invoke LLM
        print("\n📞 Calling LLM...")
        response = llm_with_tools.invoke(state["messages"])
        
        print(f"✅ LLM Response received")
        print(f"   Tool calls: {len(response.tool_calls) if response.tool_calls else 0}")
        print(f"   Content length: {len(response.content) if response.content else 0}")
        
        if response.tool_calls:
            print("\n🔧 Tool Calls Requested:")
            for tc in response.tool_calls:
                print(f"   - {tc['name']}({tc['args']})")
        
        debug_log.append({
            "node": "agent",
            "tool_calls": len(response.tool_calls) if response.tool_calls else 0,
            "response_type": "tool_call" if response.tool_calls else "text"
        })
        
        return {"messages": [response], "debug_log": debug_log}
    
    def debug_tool_node(state: DebugAgentState):
        """Tool node with debug logging"""
        debug_log = state.get("debug_log", [])
        
        print("\n" + "=" * 60)
        print("🐛 DEBUG: Tool Node Execution")
        print("=" * 60)
        
        last_message = state["messages"][-1]
        
        print(f"Tool calls to execute: {len(last_message.tool_calls)}")
        
        # Execute tools
        tool_node = ToolNode(tools)
        result = tool_node.invoke(state)
        
        print(f"✅ Tools executed: {len(result['messages'])} results")
        
        for i, msg in enumerate(result["messages"], 1):
            print(f"   {i}. {msg.content[:100]}...")
        
        debug_log.append({
            "node": "tools",
            "tools_executed": len(result["messages"]),
            "results": [str(m.content)[:50] for m in result["messages"]]
        })
        
        result["debug_log"] = debug_log
        return result
    
    # Build graph
    workflow = StateGraph(DebugAgentState)
    workflow.add_node("agent", debug_agent_node)
    workflow.add_node("tools", debug_tool_node)
    
    workflow.add_edge(START, "agent")
    workflow.add_conditional_edges("agent", tools_condition)
    workflow.add_edge("tools", "agent")
    
    return workflow.compile(checkpointer=MemorySaver())

print("✅ create_debuggable_agent function defined")

✅ create_debuggable_agent function defined


## 5.3 Test Debugging

In [45]:
debug_agent = create_debuggable_agent()

print("📝 Testing Agent with Debug Logging")
print("=" * 80)

result = debug_agent.invoke(
    {
        "messages": [HumanMessage(content="What is 100 divided by 4, and what's the weather in London?")],
        "debug_log": []
    },
    config={"configurable": {"thread_id": "debug_1"}}
)

print("\n\n" + "=" * 80)
print("📊 DEBUG LOG SUMMARY")
print("=" * 80)

for i, entry in enumerate(result["debug_log"], 1):
    print(f"\n{i}. Node: {entry['node']}")
    for key, value in entry.items():
        if key != 'node':
            print(f"   {key}: {value}")

📝 Testing Agent with Debug Logging

🐛 DEBUG: Agent Node Execution
Messages in state: 1
Last message: What is 100 divided by 4, and what's the weather in London?...

📞 Calling LLM...
✅ LLM Response received
   Tool calls: 2
   Content length: 3

🔧 Tool Calls Requested:
   - calculate({'operation': 'divide', 'a': 100, 'b': 4})
   - get_weather({'location': 'London'})

🐛 DEBUG: Tool Node Execution
Tool calls to execute: 2
🔧 Tool executed: divide(100.0, 4.0) = 25.0
🔧 Tool executed: get_weather('London') = Rainy, 52°F
✅ Tools executed: 2 results
   1. 25.0...
   2. Rainy, 52°F...

🐛 DEBUG: Agent Node Execution
Messages in state: 4
Last message: Rainy, 52°F...

📞 Calling LLM...
✅ LLM Response received
   Tool calls: 0
   Content length: 141


📊 DEBUG LOG SUMMARY

1. Node: agent
   message_count: 1
   last_message_type: HumanMessage

2. Node: agent
   tool_calls: 2
   response_type: tool_call

3. Node: tools
   tools_executed: 2
   results: ['25.0', 'Rainy, 52°F']

4. Node: agent
   message_c

---

# Bonus: Graph Visualization

In [47]:
def visualize_graph_structure(agent):
    """Display the structure of an agent graph"""
    print("\n📊 Agent Graph Structure:")
    print("-" * 60)
    
    # Get graph representation
    try:
        graph_ascii = agent.get_graph().draw_ascii()
        print(graph_ascii)
    except Exception as e:
        print(f"ASCII visualization not available: {e}")
        print("\nGraph components:")
        print(f"  Nodes: {list(agent.get_graph().nodes.keys())}")
        print(f"  Edges: Present but structure not displayable in ASCII")

visualize_graph_structure(react_agent)


📊 Agent Graph Structure:
------------------------------------------------------------
        +-----------+         
        | __start__ |         
        +-----------+         
               *              
               *              
               *              
          +-------+           
          | agent |           
          +-------+.          
          .         .         
        ..           ..       
       .               .      
+---------+         +-------+ 
| __end__ |         | tools | 
+---------+         +-------+ 


---

# Summary and Key Takeaways

## 1. ReAct Pattern
- Interleaves reasoning and acting
- Agent thinks, uses tools, observes results
- More reliable than pure prompting

## 2. Agent Nodes
- Custom logic for decision making
- State management and tracking
- Integration with multiple tools

## 3. Tool Execution
- Declarative tool definitions with `@tool` decorator
- Automatic parameter validation
- Structured inputs and outputs

## 4. Error Handling
- Retry logic for transient failures
- Error propagation to agent
- Graceful degradation

## 5. Debugging
- Comprehensive logging
- State inspection
- Execution tracing

**Remember:** Always test agents with edge cases and monitor their behavior!

---

## Next Steps

1. Try modifying the tools with your own logic
2. Experiment with different LLM models
3. Build multi-agent systems
4. Add persistence and memory
5. Deploy to production with proper monitoring